In [1]:
!pip install -qqq git+https://github.com/huggingface/diffusers.git gradio transformers accelerate safetensors

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.8 MB/s 

In [9]:
!pip install -q diffusers transformers accelerate
from diffusers import StableDiffusionInstructPix2PixPipeline
import torch

pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    "timbrooks/instruct-pix2pix", torch_dtype=torch.float16
).to("cuda")

model_index.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

safety_checker/model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
from PIL import Image
import requests, io
from IPython.display import display, clear_output
import ipywidgets as widgets
from google.colab import files
import numpy as np

# 🧩 UI Elements
method_selector = widgets.ToggleButtons(
    options=['Upload Image', 'Use URL'],
    description='Input Method:',
    button_style='info'
)

upload_button = widgets.Button(description='📤 Upload Image', button_style='primary')
url_input = widgets.Text(placeholder='Paste image URL here...', layout=widgets.Layout(width='100%'))

prompt_dropdown = widgets.Dropdown(
    options=[
        "Turn into anime character with big eyes and pastel colors",
        "Make it look like a painting with brush strokes",
        "Add sunglasses and a leather jacket",
        "Turn into futuristic robot with glowing neon armor",
        "Convert into pixel art with blocky shapes",
        "Make it look haunted with fog and glowing eyes",
        "Turn into superhero with cape and mask",
        "Make it underwater with bubbles and fish",
        "Add rainbow colors and sparkles",
        "Turn into fantasy creature with wings and horns",
        "Make it cyberpunk with neon lights and wires",
        "Convert into cartoon with exaggerated features",
        "Make it look vintage with sepia tones",
        "Add fire effects and glowing eyes",
        "Make it dreamy with soft colors and stars",
        "Turn into graffiti-style street art",
        "Add galaxy background and stars",
        "Make it cinematic with dramatic lighting",
        "Turn into sketch with pencil lines",
        "Make it look like a toy figure"
    ],
    description='Prompt:',
    layout=widgets.Layout(width='100%')
)

generate_button = widgets.Button(description='Generate ✨', button_style='success')
input_area = widgets.Output()
output_area = widgets.Output()

# 🧠 Image Loaders
def load_image_from_upload():
    uploaded = files.upload()
    if uploaded:
        try:
            img = Image.open(io.BytesIO(list(uploaded.values())[0])).convert("RGB")
            return img.resize((512, 512))
        except:
            return None
    return None

def load_image_from_url(url):
    try:
        img = Image.open(requests.get(url, stream=True).raw).convert("RGB")
        return img.resize((512, 512))
    except:
        return None

# ⚠️ Output Quality Check
def is_bad_output(img):
    arr = np.array(img)
    brightness = np.mean(arr)
    contrast = np.std(arr)
    return brightness < 20 or contrast < 10  # tweak if needed

# 🚀 Generate Handler
def on_generate_clicked(b):
    output_area.clear_output()
    with output_area:
        prompt = prompt_dropdown.value
        if not prompt:
            print("⚠️ Please select a prompt.")
            return

        if method_selector.value == 'Upload Image':
            img = load_image_from_upload()
        else:
            img = load_image_from_url(url_input.value.strip())

        if img:
            display(widgets.HTML("<b>Original Image (512×512):</b>"))
            display(img)
            display(widgets.HTML(f"<b>Transformed Image: '{prompt}'</b>"))

            try:
                edited = pipe(prompt, image=img).images[0]
                if is_bad_output(edited):
                    print("⚠️ Please try another image. This one didn’t work well because the transformation output was too dark or lacked detail.")
                else:
                    display(edited)
            except Exception as e:
                print(f"❌ Transformation failed: {str(e)}")
        else:
            print("❌ Failed to load image. Please check the URL or upload again.")

generate_button.on_click(on_generate_clicked)

# 🔄 Reactive Input Switcher
def update_input_area(change=None):
    input_area.clear_output()
    with input_area:
        if method_selector.value == 'Upload Image':
            display(upload_button)
        else:
            display(url_input)

upload_button.on_click(lambda b: on_generate_clicked(b))
method_selector.observe(update_input_area, names='value')

# 🚀 Launch UI
display(method_selector)
update_input_area()
display(input_area)
display(prompt_dropdown)
display(generate_button)
display(output_area)

ToggleButtons(button_style='info', description='Input Method:', options=('Upload Image', 'Use URL'), value='Up…

Output()

Dropdown(description='Prompt:', layout=Layout(width='100%'), options=('Turn into anime character with big eyes…

Button(button_style='success', description='Generate ✨', style=ButtonStyle())

Output()